# Capacitated Vehicle Routing Problem (CVRP)

This is a different problem from the TSP. Here we have:
- A **depot** (node P) where the trailer loads products
- A trailer with **capacity of 25 products**
- 7 delivery nodes, each requiring a specific number of products
- A **directed graph** (not all connections are bidirectional, and it's not complete)
- The trailer must return to P to restock when it runs out

Some nodes need **more than 25 products** (Node 2 needs 35, Node 7 needs 40), so the trailer must visit them multiple times (split deliveries).

Total demand: $10 + 35 + 15 + 20 + 15 + 10 + 40 = 145$ products

Minimum number of trips: $\lceil 145 / 25 \rceil = 6$ trips

In [1]:
node_names = ["P", "1", "2", "3", "4", "5", "6", "7"]
TRAILER_CAPACITY = 25

# How many products each node needs
demands = {
    "P": 0,
    "1": 10,
    "2": 35,
    "3": 15,
    "4": 20,
    "5": 15,
    "6": 10,
    "7": 40
}

total_demand = sum(demands.values())
print(f"Total demand: {total_demand} products")
print(f"Trailer capacity: {TRAILER_CAPACITY} products")
print(f"Minimum trips needed: {-(-total_demand // TRAILER_CAPACITY)}")

Total demand: 145 products
Trailer capacity: 25 products
Minimum trips needed: 6


## Directed Graph

Unlike the TSP where the graph was complete and undirected, this graph is **directed** and **sparse**. Not every node connects to every other node, and a connection from A to B doesn't guarantee a connection from B to A.

Key constraints confirmed:
- Node 3 does **not** have a direct connection to P
- Node 4 does **not** have a direct connection to Node 1

In [2]:
# Directed connections: {from_node: {to_node: distance_km}}
adj = {
    "P": {"1": 15, "2": 17, "3": 25},
    "1": {"P": 15, "3": 5, "4": 11},
    "2": {"P": 17, "7": 7, "3": 9},
    "3": {"1": 5, "4": 13, "6": 18, "7": 9},
    "4": {"3": 13, "6": 10},
    "5": {"6": 5},
    "6": {"4": 10, "3": 18, "5": 5},
    "7": {"2": 7}
}

print("Directed connections:")
for src, dests in adj.items():
    for dst, dist in dests.items():
        print(f"  {src} -> {dst}: {dist} km")

Directed connections:
  P -> 1: 15 km
  P -> 2: 17 km
  P -> 3: 25 km
  1 -> P: 15 km
  1 -> 3: 5 km
  1 -> 4: 11 km
  2 -> P: 17 km
  2 -> 7: 7 km
  2 -> 3: 9 km
  3 -> 1: 5 km
  3 -> 4: 13 km
  3 -> 6: 18 km
  3 -> 7: 9 km
  4 -> 3: 13 km
  4 -> 6: 10 km
  5 -> 6: 5 km
  6 -> 4: 10 km
  6 -> 3: 18 km
  6 -> 5: 5 km
  7 -> 2: 7 km


## Shortest Paths with Floyd-Warshall

Since the graph is not complete, to travel between two nodes that are not directly connected we need to go through intermediate nodes. We need to know the **shortest path** between every pair of nodes.

Floyd-Warshall computes all-pairs shortest paths in $O(n^3)$. For 8 nodes that's $8^3 = 512$ operations, which is trivial.

The algorithm works by considering each node as a potential intermediate stop. For every pair $(i, j)$, it checks whether going through node $k$ is shorter than the current best path:

$$d(i, j) = \min(d(i, j), \ d(i, k) + d(k, j))$$

In [3]:
INF = float('inf')
n = len(node_names)
idx = {name: i for i, name in enumerate(node_names)}

# Initialize distance and next-node matrices
dist = [[INF] * n for _ in range(n)]
next_hop = [[None] * n for _ in range(n)]

for i in range(n):
    dist[i][i] = 0

for src, dests in adj.items():
    for dst, weight in dests.items():
        i, j = idx[src], idx[dst]
        dist[i][j] = weight
        next_hop[i][j] = j

# Floyd-Warshall
for k in range(n):
    for i in range(n):
        for j in range(n):
            if dist[i][k] + dist[k][j] < dist[i][j]:
                dist[i][j] = dist[i][k] + dist[k][j]
                next_hop[i][j] = next_hop[i][k]

# Display the shortest distance matrix
header = "       " + "  ".join(f"{name:>5}" for name in node_names)
print(header)
for i, name in enumerate(node_names):
    row = f"{name:>5}  "
    for j in range(n):
        d = dist[i][j]
        row += f"{d:>5}" + "  " if d != INF else "  INF  "
    print(row)

           P      1      2      3      4      5      6      7
    P      0     15     17     20     26     41     36     24  
    1     15      0     21      5     11     26     21     14  
    2     17     14      0      9     22     32     27      7  
    3     20      5     16      0     13     23     18      9  
    4     33     18     29     13      0     15     10     22  
    5     43     28     39     23     15      0      5     32  
    6     38     23     34     18     10      5      0     27  
    7     24     21      7     16     29     39     34      0  


In [4]:
def get_shortest_distance(src, dst):
    """Return the shortest distance between two nodes."""
    return dist[idx[src]][idx[dst]]

def get_shortest_path(src, dst):
    """Reconstruct the shortest path between two nodes."""
    if next_hop[idx[src]][idx[dst]] is None:
        return None  # No path exists
    path = [src]
    current = idx[src]
    while current != idx[dst]:
        current = next_hop[current][idx[dst]]
        path.append(node_names[current])
    return path

# Verify: show some example paths
examples = [("P", "5"), ("P", "7"), ("5", "P"), ("7", "P"), ("3", "P")]
for src, dst in examples:
    path = get_shortest_path(src, dst)
    d = get_shortest_distance(src, dst)
    print(f"{src} -> {dst}: {' -> '.join(path)} = {d} km")

P -> 5: P -> 1 -> 4 -> 6 -> 5 = 41 km
P -> 7: P -> 2 -> 7 = 24 km
5 -> P: 5 -> 6 -> 3 -> 1 -> P = 43 km
7 -> P: 7 -> 2 -> P = 24 km
3 -> P: 3 -> 1 -> P = 20 km


## Greedy Nearest-Neighbor CVRP Solver

The strategy is:
1. Start at P, load 25 products
2. Find the nearest node that still needs products
3. Travel there (via shortest path), deliver as many as we can (or as many as they need)
4. If we still have products, find the next nearest unfinished node and repeat
5. When we run out of products (or all reachable nodes are served), return to P
6. Reload and repeat until all demands are met

This is a **heuristic** — it won't necessarily find the optimal solution, but it gives a reasonable one quickly.

In [5]:
def solve_greedy_nearest(demands_input):
    """Greedy nearest-neighbor CVRP solver."""
    remaining = {node: d for node, d in demands_input.items()}
    total_distance = 0
    trips = []

    while any(remaining[node] > 0 for node in node_names if node != "P"):
        # Start a new trip from P
        capacity = TRAILER_CAPACITY
        current = "P"
        trip_path = ["P"]
        trip_dist = 0
        trip_deliveries = []

        while capacity > 0:
            # Find nearest node with remaining demand
            best_node = None
            best_dist = INF

            for node in node_names:
                if node == "P" or remaining[node] <= 0:
                    continue
                d = get_shortest_distance(current, node)
                if d < best_dist:
                    best_dist = d
                    best_node = node

            if best_node is None:
                break  # All demands fulfilled

            # Travel to the best node
            path_segment = get_shortest_path(current, best_node)
            trip_path.extend(path_segment[1:])  # Skip current (already in path)
            trip_dist += best_dist

            # Deliver products
            deliver = min(capacity, remaining[best_node])
            remaining[best_node] -= deliver
            capacity -= deliver
            trip_deliveries.append((best_node, deliver))
            current = best_node

        # Return to P
        return_path = get_shortest_path(current, "P")
        trip_path.extend(return_path[1:])
        trip_dist += get_shortest_distance(current, "P")
        total_distance += trip_dist

        trips.append({
            "path": trip_path,
            "distance": trip_dist,
            "deliveries": trip_deliveries
        })

    return trips, total_distance


def print_solution(trips, total_distance, strategy_name):
    """Display the solution nicely."""
    print(f"=== {strategy_name} ===")
    print(f"Total trips: {len(trips)}")
    print(f"Total distance: {total_distance} km\n")

    for i, trip in enumerate(trips, 1):
        products_delivered = sum(d[1] for d in trip['deliveries'])
        print(f"Trip {i}: {' -> '.join(trip['path'])}")
        print(f"  Distance: {trip['distance']} km")
        print(f"  Deliveries: ", end="")
        print(", ".join(f"Node {d[0]}: {d[1]} products" for d in trip['deliveries']))
        print(f"  Products delivered this trip: {products_delivered}")
        print()

In [6]:
trips_nearest, dist_nearest = solve_greedy_nearest(demands)
print_solution(trips_nearest, dist_nearest, "Greedy Nearest-Neighbor")

=== Greedy Nearest-Neighbor ===
Total trips: 6
Total distance: 328 km

Trip 1: P -> 1 -> 3 -> 1 -> P
  Distance: 40 km
  Deliveries: Node 1: 10 products, Node 3: 15 products
  Products delivered this trip: 25

Trip 2: P -> 2 -> P
  Distance: 34 km
  Deliveries: Node 2: 25 products
  Products delivered this trip: 25

Trip 3: P -> 2 -> 7 -> 2 -> P
  Distance: 48 km
  Deliveries: Node 2: 10 products, Node 7: 15 products
  Products delivered this trip: 25

Trip 4: P -> 2 -> 7 -> 2 -> P
  Distance: 48 km
  Deliveries: Node 7: 25 products
  Products delivered this trip: 25

Trip 5: P -> 1 -> 4 -> 6 -> 3 -> 1 -> P
  Distance: 74 km
  Deliveries: Node 4: 20 products, Node 6: 5 products
  Products delivered this trip: 25

Trip 6: P -> 1 -> 4 -> 6 -> 5 -> 6 -> 3 -> 1 -> P
  Distance: 84 km
  Deliveries: Node 6: 5 products, Node 5: 15 products
  Products delivered this trip: 20



## Alternative Strategies

The nearest-neighbor heuristic is simple but can be short-sighted. Let's try two other strategies:

1. **Highest-demand-first**: prioritize nodes that need the most products (reduces future trips to far nodes)
2. **Farthest-first**: deliver to far nodes first while the trailer is full, handle nearby nodes on shorter trips

In [7]:
def solve_greedy(demands_input, pick_strategy):
    """Generic greedy CVRP solver with pluggable node-selection strategy."""
    remaining = {node: d for node, d in demands_input.items()}
    total_distance = 0
    trips = []

    while any(remaining[node] > 0 for node in node_names if node != "P"):
        capacity = TRAILER_CAPACITY
        current = "P"
        trip_path = ["P"]
        trip_dist = 0
        trip_deliveries = []

        while capacity > 0:
            candidates = [
                node for node in node_names
                if node != "P" and remaining[node] > 0
            ]
            if not candidates:
                break

            best_node = pick_strategy(current, candidates, remaining)

            path_segment = get_shortest_path(current, best_node)
            trip_path.extend(path_segment[1:])
            trip_dist += get_shortest_distance(current, best_node)

            deliver = min(capacity, remaining[best_node])
            remaining[best_node] -= deliver
            capacity -= deliver
            trip_deliveries.append((best_node, deliver))
            current = best_node

        return_path = get_shortest_path(current, "P")
        trip_path.extend(return_path[1:])
        trip_dist += get_shortest_distance(current, "P")
        total_distance += trip_dist

        trips.append({
            "path": trip_path,
            "distance": trip_dist,
            "deliveries": trip_deliveries
        })

    return trips, total_distance


# Strategy 1: pick the node with the highest remaining demand
def pick_highest_demand(current, candidates, remaining):
    return max(candidates, key=lambda node: remaining[node])

# Strategy 2: pick the farthest node from current position
def pick_farthest(current, candidates, remaining):
    return max(candidates, key=lambda node: get_shortest_distance(current, node))

# Strategy 3: nearest neighbor (same as before, for comparison)
def pick_nearest(current, candidates, remaining):
    return min(candidates, key=lambda node: get_shortest_distance(current, node))


strategies = [
    ("Nearest-Neighbor", pick_nearest),
    ("Highest-Demand-First", pick_highest_demand),
    ("Farthest-First", pick_farthest),
]

In [8]:
results = []

for name, strategy in strategies:
    trips, total_dist = solve_greedy(demands, strategy)
    results.append((name, trips, total_dist))
    print_solution(trips, total_dist, name)
    print("=" * 60 + "\n")

# Summary comparison
print("\n" + "=" * 40)
print("STRATEGY COMPARISON")
print("=" * 40)
for name, trips, total_dist in sorted(results, key=lambda x: x[2]):
    print(f"{name:.<30} {total_dist:>6} km  ({len(trips)} trips)")

best = min(results, key=lambda x: x[2])
worst = max(results, key=lambda x: x[2])
print(f"\nBest strategy: {best[0]} with {best[2]} km")
print(f"Savings vs worst: {worst[2] - best[2]} km")

=== Nearest-Neighbor ===
Total trips: 6
Total distance: 328 km

Trip 1: P -> 1 -> 3 -> 1 -> P
  Distance: 40 km
  Deliveries: Node 1: 10 products, Node 3: 15 products
  Products delivered this trip: 25

Trip 2: P -> 2 -> P
  Distance: 34 km
  Deliveries: Node 2: 25 products
  Products delivered this trip: 25

Trip 3: P -> 2 -> 7 -> 2 -> P
  Distance: 48 km
  Deliveries: Node 2: 10 products, Node 7: 15 products
  Products delivered this trip: 25

Trip 4: P -> 2 -> 7 -> 2 -> P
  Distance: 48 km
  Deliveries: Node 7: 25 products
  Products delivered this trip: 25

Trip 5: P -> 1 -> 4 -> 6 -> 3 -> 1 -> P
  Distance: 74 km
  Deliveries: Node 4: 20 products, Node 6: 5 products
  Products delivered this trip: 25

Trip 6: P -> 1 -> 4 -> 6 -> 5 -> 6 -> 3 -> 1 -> P
  Distance: 84 km
  Deliveries: Node 6: 5 products, Node 5: 15 products
  Products delivered this trip: 20


=== Highest-Demand-First ===
Total trips: 6
Total distance: 390 km

Trip 1: P -> 2 -> 7 -> 2 -> P
  Distance: 48 km
  Deliver

## What makes this different from the TSP?

| Feature | TSP | CVRP |
|---------|-----|------|
| Graph | Complete, undirected | Sparse, directed |
| Visit each node | Exactly once | Possibly multiple times (split deliveries) |
| Capacity constraint | None | 25 products per trip |
| Return to depot | Once at the end | Multiple times to restock |
| Brute force feasible? | $(n-1)!/2$ paths | State space is much larger due to split deliveries |

The CVRP is NP-hard just like the TSP, but the capacity constraint and split deliveries make the solution space significantly more complex. Heuristics like the ones above are the standard approach for practical instances.

## Comparing with the Manual Solution

Your classmates got 24,225 MXN (323 km at 75 MXN/km). Let's verify their routes step by step against the actual graph distances, including the **return trip to P** which they may have miscalculated on some legs.

In [9]:
def trace_trip(stops, deliveries_dict, label):
    """Trace a trip through the graph and compute actual distance."""
    total_dist = 0
    full_path = [stops[0]]
    segments = []
    
    for i in range(len(stops) - 1):
        seg_dist = get_shortest_distance(stops[i], stops[i+1])
        seg_path = get_shortest_path(stops[i], stops[i+1])
        segments.append((stops[i], stops[i+1], seg_dist, seg_path))
        total_dist += seg_dist
        full_path.extend(seg_path[1:])
    
    # Return to P
    last = stops[-1]
    ret_dist = get_shortest_distance(last, "P")
    ret_path = get_shortest_path(last, "P")
    segments.append((last, "P", ret_dist, ret_path))
    total_dist += ret_dist
    full_path.extend(ret_path[1:])
    
    products = sum(deliveries_dict.values())
    
    print(f"\n{label}")
    print(f"  Path: {' → '.join(full_path)}")
    for src, dst, d, path in segments:
        via = f" (via {' → '.join(path)})" if len(path) > 2 else ""
        print(f"    {src} → {dst}: {d} km{via}")
    print(f"  Deliveries: {', '.join(f'Node {k}: {v}' for k, v in deliveries_dict.items())}")
    print(f"  Products: {products}/25  |  ACTUAL distance: {total_dist} km")
    
    return total_dist


# Define classmates' trips: (label, delivery_stops, {node: products}, claimed_km)
classmate_trips = [
    ("Trip a: P→1→4→6→5→P",   ["P", "1", "4", "6", "5"], {"6": 10, "5": 15},            82),
    ("Trip b: P→1→4→P",        ["P", "1", "4"],            {"1": 5, "4": 20},              52),
    ("Trip c: P→1→3→7→P",      ["P", "1", "3", "7"],       {"1": 5, "3": 15, "7": 5},      59),
    ("Trip d: P→2→7→P",        ["P", "2", "7"],            {"7": 25},                       48),
    ("Trip e: P→2→7→P",        ["P", "2", "7"],            {"2": 15, "7": 10},              48),
    ("Trip f: P→2→P",          ["P", "2"],                 {"2": 20},                       34),
]

print("=" * 60)
print("VERIFYING CLASSMATES' MANUAL SOLUTION")
print("=" * 60)

total_claimed = 0
total_actual = 0
delivery_tally = {}

for label, stops, deliveries, claimed in classmate_trips:
    actual = trace_trip(stops, deliveries, label)
    diff = actual - claimed
    marker = "✓" if diff == 0 else f"✗ (off by {diff:+d} km)"
    print(f"  Claimed: {claimed} km  |  {marker}")
    total_claimed += claimed
    total_actual += actual
    for node, qty in deliveries.items():
        delivery_tally[node] = delivery_tally.get(node, 0) + qty

# Verify all demands met
print(f"\n{'=' * 60}")
print("DELIVERY CHECK")
print("=" * 60)
for node in ["1", "2", "3", "4", "5", "6", "7"]:
    delivered = delivery_tally.get(node, 0)
    needed = demands[node]
    status = "✓" if delivered == needed else f"✗ (short {needed - delivered})"
    print(f"  Node {node}: {delivered}/{needed} {status}")

# Final comparison
COST_PER_KM = 75
print(f"\n{'=' * 60}")
print("FINAL COMPARISON")
print("=" * 60)
print(f"  Classmates claimed:    {total_claimed} km  →  {total_claimed * COST_PER_KM:,} MXN")
print(f"  Classmates ACTUAL:     {total_actual} km  →  {total_actual * COST_PER_KM:,} MXN")
print(f"  Algorithm (nearest):   {dist_nearest} km  →  {dist_nearest * COST_PER_KM:,} MXN")
print(f"\n  Classmates' routes are actually {dist_nearest - total_actual} km shorter than the algorithm.")

VERIFYING CLASSMATES' MANUAL SOLUTION

Trip a: P→1→4→6→5→P
  Path: P → 1 → 4 → 6 → 5 → 6 → 3 → 1 → P
    P → 1: 15 km
    1 → 4: 11 km
    4 → 6: 10 km
    6 → 5: 5 km
    5 → P: 43 km (via 5 → 6 → 3 → 1 → P)
  Deliveries: Node 6: 10, Node 5: 15
  Products: 25/25  |  ACTUAL distance: 84 km
  Claimed: 82 km  |  ✗ (off by +2 km)

Trip b: P→1→4→P
  Path: P → 1 → 4 → 3 → 1 → P
    P → 1: 15 km
    1 → 4: 11 km
    4 → P: 33 km (via 4 → 3 → 1 → P)
  Deliveries: Node 1: 5, Node 4: 20
  Products: 25/25  |  ACTUAL distance: 59 km
  Claimed: 52 km  |  ✗ (off by +7 km)

Trip c: P→1→3→7→P
  Path: P → 1 → 3 → 7 → 2 → P
    P → 1: 15 km
    1 → 3: 5 km
    3 → 7: 9 km
    7 → P: 24 km (via 7 → 2 → P)
  Deliveries: Node 1: 5, Node 3: 15, Node 7: 5
  Products: 25/25  |  ACTUAL distance: 53 km
  Claimed: 59 km  |  ✗ (off by -6 km)

Trip d: P→2→7→P
  Path: P → 2 → 7 → 2 → P
    P → 2: 17 km
    2 → 7: 7 km
    7 → P: 24 km (via 7 → 2 → P)
  Deliveries: Node 7: 25
  Products: 25/25  |  ACTUAL distance: 

### Analysis

Your classmates' **route choices are actually better** than the greedy algorithm — their actual total is **326 km** (24,450 MXN), which beats the algorithm's 328 km (24,600 MXN) by 2 km.

However, their **arithmetic has some errors**:
- **Trip a** (P→1→4→6→5→P): they forgot to count the full return from node 5 back to P, which goes 5→6→3→1→P = 43 km. Actual is 84, not 82. (**+2 km**)
- **Trip b** (P→1→4→P): the return from node 4 goes 4→3→1→P = 33 km. Actual is 59, not 52. (**+7 km**)  
- **Trip c** (P→1→3→7→P): the return from 7 goes 7→2→P = 24 km, making the actual trip 53 km, not 59. They actually **overestimated** this one. (**-6 km**)

The errors partially cancel out (+2 +7 -6 = +3 km), so their claimed 323 km is actually 326 km.

**Why did the greedy algorithm lose?** Because nearest-neighbor is short-sighted: it always picks the closest node next, which can lead to inefficient return trips later. The classmates made smarter strategic choices — like combining node 6 and 5 deliveries into one deep trip (trip a), even though those nodes are far from P. 

The greedy algorithm is a heuristic, not an exact solver. For small problems like this, thoughtful manual planning can beat it!